In [2]:
import os, cv2
import numpy as np

def read_anno_file(anno_file):
    assert os.path.exists(anno_file), "Annotation file does not exist!" + anno_file
    result = []
    with open(anno_file, 'r') as f:
        for line in f.readlines():
            items = {}
            items['vid'] = line.strip().split(',[')[0]
            labels = line.strip().split(',[')[1].split('],')[0]
            items['label'] = [int(val) for val in labels.split(',')]
            others = line.strip().split(',[')[1].split('],')[1].split(',')
            items['startframe'], items['vid_ytb'], items['lighting'], items['weather'], items['ego_involve'] = others
            result.append(items)
    f.close()
    return result


def get_video_frames(video_file, topN=50):
    # get the video data
    cap = cv2.VideoCapture(video_file)
    ret, frame = cap.read()
    video_data = []
    while (ret):
        video_data.append(frame)
        ret, frame = cap.read()
    print("original # frames: %d"%(len(video_data)))
    assert len(video_data) >= topN
    video_data = video_data[:topN]
    return video_data



In [ ]:

if __name__ == "__main__":
    anno_file = "../videos/Crash-1500.txt"
    anno_data = read_anno_file(anno_file)

    video_path = "../videos/Crash-1500"
    for anno in anno_data:
        video_file = os.path.join(video_path, anno['vid'] + ".mp4")
        assert os.path.exists(video_file), "video file does not exist!" + video_file
        # read frames
        frames = get_video_frames(video_file, topN=50)
        labels = anno['label']
        print("file: %s, # frames: %d, # labels: %d."%(video_file, len(frames), len(labels)))
        print(len(labels))
        for idx, im in enumerate(frames):
           if labels[idx] == 1:
               cv2.putText(im, 'Accident', (int(im.shape[1] / 2)-60, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2, cv2.LINE_AA)
           else:
               cv2.putText(im, 'Normal', (int(im.shape[1] / 2)-60, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 2, cv2.LINE_AA)
           cv2.imshow('frame', im)
           cv2.waitKey(100)

In [4]:
annotation = read_anno_file("/Users/lucabankofski/Documents/ECE/Cours/PFE_RECHERCHE_VOITURE/Dataset/CarCrash/videos/Crash-1500.txt")

In [5]:
import pandas as pd
annotation_df=pd.DataFrame(annotation)

la colonne label contiens 50 label qui sont des booléens. 1 donne la frame ou se situe l'accident.

In [6]:
annotation_df.head()

,vid,label,startframe,vid_ytb,lighting,weather,ego_involve
0,000001,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",000285,0000,Day,Normal,Yes
1,000002,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",000646,0000,Day,Normal,Yes
2,000003,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",000972,0000,Day,Normal,Yes
3,000004,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",001257,0000,Day,Normal,Yes
4,000005,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",000462,0010,Day,Snowy,Yes


In [10]:
def read_features_file(file):
    #Code to open and read feature in npz files
    features = np.load(file)    
    return features
    

In [12]:
np_file=read_features_file("/Users/lucabankofski/Documents/ECE/Cours/PFE_RECHERCHE_VOITURE/Dataset/CarCrash/vgg16_features/positive/000004.npz")

- data: Extracted 4096-dim features with shape (50, 20, 4096). It contains frame-level feature with shape (50, 1, 4096) and 19 box-level features with shape (50, 19, 4096).
- det: Detected bounding boxes with shape (50, 19, 6), where the last dim denotes (x1, y1, x2, y2, prob, cls).
- labels: One-hot video labels to indicate whether the video contains an accident, i.e., [0, 1] denotes positive (accident) and [1, 0] denotes negative (normal).
- ID: The video name for current feature file.


In [15]:
np_file.files

['data', 'det', 'labels', 'ID']

In [19]:
np_file["ID"]

array('000004', dtype='<U6')